In [21]:
import time
import os
import math
import numpy as np
import pandas as pd
import pyautogui as pa
import shutil
import imageio
from PIL import Image, ImageDraw
import random

from lib.auto_GUI.auto_GUI_base import AutoGUIBase
from lib.auto_GUI.auto_DAT import AutoDAT
from lib.auto_GUI.auto_PhotoZ import AutoPhotoZ
from lib.analysis.laminar_dist import *
from lib.file.TIF import *
from lib.analysis.align import ImageAlign
from lib.utilities import *
from lib.file.ROI_writer import ROIFileWriter

###########################################
# Make a series of 1-frame measure windows from PhotoZ into a movie
# Asks user to annotate DIC/Fluorescence images for electrode + layer and barrel boundaries
#       Puts this metadata into ROI .dat files
# Uses DIC-to-ZDA frame mapping (rotation and scaling) to annotate the PhotoZ frames
# Uses autoPhotoZ to take frame images for move
#  - 8/24/23: 
#        Support any number of barrels.
#        add ROI generation: layer/barrel ROIs
###########################################

In [22]:
# Load data, not from Drive for Desktop since PhotoZ complains about loading zda from Drive
data_dir = "C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Anna-NBQX/"
start_frame = 94
movie_length = 200  # frames
frame_step_size = 4
camera_program = 4
which_rig = 'new'  # change to 'old' for old rig dic alignment
date_format = 'yy-mm-dd'
if which_rig == 'old':
    date_format = 'yyyy-mm-dd'

# annotation settings
skip_draw_annotations = True  # If True, don't draw any new annotations
disable_annotations = True  # If True, don't draw any annotations

# Movie maker settings
disable_photoZ_interact = False
initialize_photoZ = False
clean_scratch_files = False  # clean up SNR maps
replace_existing_frame_pngs = False
disable_google_drive_interact = True
image_border_width = 8
image_scaling = 7  # pixels of image per array diode pixel
electrode_marker_radius = 4

In [23]:
# set up PhotoZ (open it manually)
aPhz = AutoPhotoZ(data_dir=data_dir)
if initialize_photoZ and not disable_photoZ_interact:
    aPhz.prepare_photoZ()
if not disable_photoZ_interact:
    pa.alert("Please disable trace overlay & normalization, and maximize PhotoZ before continuing.")

In [24]:
# run this cell to refresh and re-do all processing.
rec_id_skip_list = []

In [25]:
# First ask user to annotate some images with Electrode loc and L4 boundaries and layer text placement
# based on DIC-to-ZDA_Alignment script
for subdir, dirs, files in os.walk(data_dir):
    if disable_annotations:
        continue
    if 'notUsable' in subdir:
        continue
    if 'hidden' in subdir:
        continue
    if 'dic' in dirs and 'selected_zda' in dirs:
        print("\n", subdir)
        dic_dir = subdir + "/dic/"
        selected_zda_dir = subdir + "/selected_zda/"
        already_drawn_slic_loc = {}
        
        # optionally process date
        date = ""
        try:
            char_select = -len(date_format)
            date = subdir.split("_Usable")[0][char_select:]
            date = [int(x) for x in date.split("-")]
            if date_format != 'yyyy-mm-dd':
                date[2] += 2000  # full year format
            else:
                date = [date[1], date[2], date[0]]
            date = "/".join([str(d) for d in date])
        except Exception as e:
            print(e, "could not process date.")

        # take selected zda and expand into separate subdir for each zda file
        for zda_file in os.listdir(selected_zda_dir):
            if zda_file.endswith('.zda'):
                rec_id = zda_file.split('.')[0]
                if subdir+rec_id in rec_id_skip_list:
                    continue
                print("\n", subdir)
                slic_id, loc_id, _ = [int(x) for x in rec_id.split("_")]
                # output dir
                output_dir = selected_zda_dir + "/analysis" + rec_id + "/"
                try:
                    os.makedirs(output_dir)
                except Exception as e:
                    pass

                # read in 8-bit single TIF images to array, apply same cropping/binning
                image_data = {}
                cam_settings = CameraSettings().get_program_settings(camera_program)
                binning = int(2048 / 400) # if want size similar to RLI
                TIFLoader(dic_dir, 
                          cam_settings, 
                          binning, 
                          crop=False, 
                          flip_horiz=True).load_files(image_data, 
                                                      slice_target=slic_id)

                # align RLI and DIC and record the RLI's image boundaries within the DIC image.
                img_aligner = ImageAlign(rig=which_rig)

                for slic in image_data:
                    for loc in image_data[slic]:
                        if slic != slic_id or loc != loc_id:
                            continue
                        print(slic, loc)

                        # if already drawn, no need to ask user again
                        if str(slic_id)+"_"+str(loc_id) in already_drawn_slic_loc:
                            # just copy them over
                            src_dir = already_drawn_slic_loc[str(slic_id)+"_"+str(loc_id)]
                            electrode_file = src_dir + "electrode.dat"
                            corners_layer_file = src_dir + "corners_layer.dat"
                            corners_barrel_file = src_dir + "corners_barrel.dat"
                            shutil.copy(electrode_file, output_dir + "electrode.dat")
                            shutil.copy(corners_layer_file, output_dir + "corners_layer.dat")
                            shutil.copy(corners_barrel_file, output_dir + "corners_barrel.dat")
                            print("Copied previous annotations over.")
                            continue

                        # DICs
                        fluor = None
                        if 'f' in image_data[slic][loc]:
                            fluor = image_data[slic][loc]['f']
                        elif 'fe' in image_data[slic][loc]:
                            fluor = image_data[slic][loc]['fe']
                        # img = image_data[slic][loc]['i']

                        dic_electrode = None
                        if 'e' in image_data[slic][loc]:
                            dic_electrode = image_data[slic][loc]['e']
                        else:
                            dic_electrode = fluor
                            
                        if which_rig != 'new':
                            fluor = dic_electrode

                        dic_electrode = np.array(dic_electrode, dtype=np.uint8)
                        orig_arr_shape = dic_electrode.shape

                        # ask user to select site of stim and layer/barrel borders
                        electrode_file = output_dir + "electrode.dat"
                        corners_layer_file = output_dir + "corners_layer.dat"
                        corners_barrel_file = output_dir + "corners_barrel.dat"
                        if not skip_draw_annotations:
                            dic_electrode, coords_electrode, dic_layers, coords_layers = \
                                img_aligner.draw_on_images_wrapper(
                                    dic_electrode, 
                                    fluor, 
                                    date + " " + rec_id + " ")
                            dic_electrode = np.array(dic_electrode)
                            dic_layers = np.array(dic_layers)
                            
                            plt.imshow(dic_electrode, cmap='gray')
                            x_el, y_el = coords_electrode['electrode']
                            plt.plot(x_el, y_el, marker="*", color='red')
                            dcs = []
                            if which_rig != 'new':  # show the portion of frame which is PhotoZ
                                photoZ_frame_coords = img_aligner.dic_coordinates
                                for dc_c in photoZ_frame_coords:
                                    x_c = dc_c[0] * dic_electrode.shape[1]
                                    y_c = dc_c[1] * dic_electrode.shape[0]
                                    dcs.append([x_c, y_c])
                                print(dcs)
                                for ic, jc in [[0,1],[1,3],[3,2],[2,0]]:
                                    plt.plot([dcs[ic][0], dcs[jc][0]], [dcs[ic][1], dcs[jc][1]], 
                                             color="white", 
                                             linewidth=3)
                            plt.show()

                            plt.imshow(dic_layers, cmap='gray')
                            for key_pt in coords_layers:
                                if "_axis" in key_pt:
                                    for pt in coords_layers[key_pt]:
                                        x, y = pt
                                        plt.plot(x, y, marker="*", color='green')
                            if which_rig != 'new':  # show the portion of frame which is PhotoZ
                                photoZ_frame_coords = img_aligner.dic_coordinates
                                for ic, jc in [[0,1],[1,3],[3,2],[2,0]]:
                                    plt.plot([dcs[ic][0], dcs[jc][0]], [dcs[ic][1], dcs[jc][1]], 
                                             color="white", 
                                             linewidth=3)
                            plt.show()

                            try:
                                os.makedirs(output_dir)
                            except OSError:
                                pass

                            # transform coordinates -- Actual alignment work done here
                            print("\n \tcoords_electrode", coords_electrode)
                            print("\tcoords_layers", coords_layers)
                            coords_electrode = img_aligner.transform_from_dic_coordinates(coords_electrode, orig_arr_shape)
                            coords_layers = img_aligner.transform_from_dic_coordinates(coords_layers, orig_arr_shape)
                            coordinates = coords_electrode | coords_layers

                            # write electrode and corners to file
                            img_aligner.write_shapes_to_files(coordinates,
                                                              electrode_file,
                                                              corners_layer_file,
                                                              corners_barrel_file,
                                                              preserve_initial_order=True)
                            print("Wrote files to", output_dir)

                        ############################################
                        # Create ROIs for each layer/barrel division
                        ############################################
                        # read barrel and layer axes data back in
                        # create collections of pixels by LAYER so we can create separate files for each LAYER'S barrels
                        barrel_layer_axes = []  # going to be a list of Line objects, first 2 are layer boundaries, rest are barrel boundaries
                        for la_file in [corners_layer_file, corners_barrel_file]:
                            with open(la_file, 'r') as f:
                                lines = f.readlines() 
                            roi_px_ct = int(lines[2]) - 1 
                            corners = [int(x) for x in lines[4:]] # the last lines are diode numbers of corners
                            if len(corners) != roi_px_ct:
                                raise Exception("Likely an error processing corners. Expected " + 
                                                str(roi_px_ct) + " pixels.")
                            layer_axes = LayerAxes(corners[:4], verbose=False, obey_initial_order=True)
                            laminar_axis, laminar_axis_2 = layer_axes.get_layer_axes()
                            barrel_layer_axes.append(laminar_axis)
                            barrel_layer_axes.append(laminar_axis_2)

                            rest_of_corners = LaminarROI(corners[4:]).get_points()
                            for j in range(0, len(rest_of_corners), 2):
                                if len(rest_of_corners) > j+1:
                                    barrel_layer_axes.append(Line(rest_of_corners[j], rest_of_corners[j+1]))

                        # Now we will divide into L2/3, L4, L5 and barrels
                        w, h = LaminarROI([]).get_dimensions()
                        
                        # first two are guaranteed to be LAYER divisions
                        layer_4_list = {}
                        layer_axis_1 = barrel_layer_axes[0]
                        layer_axis_2 = barrel_layer_axes[1]
                        px_map_1 = layer_axis_1.get_split_px_map() 
                        px_map_2 = layer_axis_2.get_split_px_map()
                        px_map = px_map_1 + px_map_2 * 2
                        layer_4_px_map = np.logical_xor(px_map_1, px_map_2)
                        
                        # find the number of connected domains in layer_4_px_map
                        if not Line.has_one_connected_domain(layer_4_px_map):
                            print("Flipping L4 map!")
                            layer_4_px_map = np.ones(layer_4_px_map.shape) - layer_4_px_map
                        
                        # convert layer 4 px map to list format
                        for i in range(w):
                            for j in range(h):
                                if layer_4_px_map[i,j] > 0:
                                    layer_4_list[str(i) + "_" + str(j)] = [i, j]

                        n_unique_rois = np.unique(px_map)  # should match the number of ROIs expected, which is 3 so far
                        print("PX_MAP should be divided into 3 layers at this point. " +
                                            "Number of unique layers found: " + str(len(n_unique_rois)) )
                        
                        # Now divide into ROIs
                        incrementer = 4
                        for axis in barrel_layer_axes[2:]: # rest of these are barrel divisions
                            for i in range(w):
                                for j in range(h):
                                    if axis.is_point_left_of_line([i, j]):
                                        px_map[i, j] += incrementer
                            incrementer *= 2
                            
                        n_unique_rois = np.unique(px_map)  # should match the number of ROIs expected
                        roi_id_map = {}
                        roi_layer_4_map = {}
                        layer4_idxs = {}
                        for i in range(len(n_unique_rois)):
                            roi_id_map[n_unique_rois[i]] = i

                        plt.clf()
                        plt.imshow(np.transpose(px_map))
                        for axis in barrel_layer_axes:
                            p1, p2 = axis.get_line_repr()
                            plt.plot([p1[0], p2[0]], [p1[1], p2[1]], 'ro', linestyle="--")
                            plt.plot(p1[0], p1[1], marker="*", color='green')
                            plt.plot(p2[0], p2[1], marker="*", color='blue')
                        for k in layer_4_list:
                            if random.random() > 0.95:
                                l4_pt = layer_4_list[k]
                                plt.plot(l4_pt[0], l4_pt[1], marker=".", color='red', alpha=0.3)
                        plt.show()

                        # Then we will take this Pixel classification map and turn into ROIs
                        roi_list = [[] for _ in range(len(n_unique_rois))]  # doubly-nested list of regions
                        roi_writer = ROIFileWriter()
                        roi_creater = ROICreator(None)
                        roi_output_file = selected_zda_dir + "layers_barrels_slice" + str(slic) + "_" + str(loc) +"_rois.dat"
                        for i in range(w):
                            for j in range(h):
                                roi_idx = roi_id_map[px_map[i, j]]
                                dn = roi_creater.convert_point_to_diode_number([i, j])
                                roi_list[roi_idx].append(dn)
                        roi_writer.write_regions_to_dat(roi_output_file, roi_list)
                        
                        # now write a separate file with only layer 4 rois
                        l4_roi_list = [[] for _ in range(len(n_unique_rois))]
                        l4_roi_output_file = selected_zda_dir + "layer4_barrels_slice" + str(slic)  + "_" + str(loc) +"_rois.dat"
                        for i in range(w):
                            for j in range(h):
                                if str(i) + "_" + str(j) in layer_4_list:
                                    roi_idx = roi_id_map[px_map[i, j]]
                                    dn = roi_creater.convert_point_to_diode_number([i, j])
                                    l4_roi_list[roi_idx].append(dn)
                        # filter empty ROIs
                        l4_roi_list = [r for r in l4_roi_list if len(r) > 0]
                        roi_writer.write_regions_to_dat(l4_roi_output_file, l4_roi_list)

                        # mark this slice/loc already drawn 
                        already_drawn_slic_loc[str(slic_id)+"_"+str(loc_id)] = output_dir
                image_data.clear()
                rec_id_skip_list.append(subdir+rec_id)

In [26]:
current_color_bound_setting = 1.0
for subdir, dirs, files in os.walk(data_dir):
    if 'notUsable' in subdir:
        continue
    if 'hidden' in subdir:
        continue
    if 'dic' in dirs and 'selected_zda' in dirs:
        print(subdir)
        dic_dir = subdir + "/dic/"
        selected_zda_dir = subdir + "/selected_zda/"

        for zda_file in os.listdir(selected_zda_dir):
            if zda_file.endswith('.zda'):

                rec_id = zda_file.split('.')[0]
                print(rec_id)
                # movie dir
                output_dir = selected_zda_dir + "/analysis" + rec_id + "/"
                try:
                    os.makedirs(output_dir)
                except Exception as e:
                    pass

                # determine if we are even missing any jpg'd frames
                need_to_open_zda = True
                for frame in range(start_frame, start_frame + movie_length, frame_step_size):
                    filename = output_dir + str(frame) + ".jpg"
                    if not os.path.exists(filename):
                        need_to_open_zda = True
                        break
                if not need_to_open_zda and not replace_existing_frame_pngs:
                    continue

                recording_max_snr = 3.0  # gets changed

                if not disable_photoZ_interact:
                    aPhz = AutoPhotoZ(data_dir=selected_zda_dir)
                    # open the PhotoZ file
                    aPhz.select_PhotoZ()

                    print("\n\nOpening", zda_file)
                    aPhz.open_zda_file(selected_zda_dir + "/" + zda_file)
                    time.sleep(13)

                    # estimate the SNR max across all frames
                    aPhz.set_measure_window(start_frame, movie_length, sleep_time_window_change=17)
                    filename = aPhz.save_background()
                    print("Saved intermed_snr_df file:", filename)
                    if filename is not None:  # otherwise, sticks with default rec max

                        # open Data.dat and read in the array max
                        intermed_snr_df = pd.read_csv(filename, 
                                                      sep='\t', 
                                                      header=None, 
                                                      names=['Index',  'Values'])
                        recording_max_snr = intermed_snr_df['Values'].max()

                        recording_max_snr *= 1.05  # 5% higher for headroom
                        recording_max_snr = round(recording_max_snr, 2)
                        if clean_scratch_files:
                            try:
                                os.remove(filename)
                            except Exception as e:
                                print("Could not remove", filename)
                                print(e)

                    # set the normalization to be consistent across entire video
                    aPhz.set_color_upper_bound(recording_max_snr, current_color_bound_setting)
                    print("Moved color upper bound from", current_color_bound_setting, "to", recording_max_snr)
                    current_color_bound_setting = recording_max_snr

                    # set measure window width to 1
                    aPhz.set_measure_window(None, frame_step_size, sleep_time_window_change=17)

                ##### Get frame annotation: layer and stim markings, and store #####

                # file conventions across
                electrode_file = output_dir + "/electrode.dat"
                corners_layer_file = output_dir + "/corners_layer.dat"
                corners_barrel_file = output_dir + "/corners_barrel.dat"
                annotations_enabled = all([os.path.exists(f) for f in [electrode_file, 
                                                                       corners_layer_file, 
                                                                       corners_barrel_file]])

                def array_point_to_image_pixel(arr_pt):
                    x_arr, y_arr = arr_pt
                    return [image_border_width + image_scaling * x_arr, 
                            image_border_width + image_scaling * y_arr]

                if annotations_enabled:
                    # read in electrode location
                    with open(electrode_file, 'r') as f:
                        lines = f.readlines()
                    stim_pt = int(lines[-1]) # last line is always electrode location
                    aux_obj = LaminarROI([stim_pt]).get_points()
                    stim_pt = aux_obj[0]  # should be a list of len 2, representing px location [x, y]
                    print("Stim point:", stim_pt)
                    stim_img_px = array_point_to_image_pixel(stim_pt)
                    stim_xy = (stim_img_px[0] - electrode_marker_radius, 
                               stim_img_px[1] - electrode_marker_radius, 
                               stim_img_px[0] + electrode_marker_radius, 
                               stim_img_px[1] + electrode_marker_radius)
                    # read in corners files
                    la_corners = {corners_layer_file: {},
                                  corners_barrel_file: {}}
                    for la_file in [corners_layer_file, corners_barrel_file]:
                        with open(la_file, 'r') as f:
                            lines = f.readlines() 
                        corners = [int(x) for x in lines[4:]] # the last 4 lines are diode numbers of corners
                        layer_axes = LayerAxes(corners, obey_initial_order=True)
                        laminar_axis, laminar_axis_2 = layer_axes.get_layer_axes()
                        print("laminar axis unit vectors:", laminar_axis.get_unit_vector(), 
                              laminar_axis_2.get_unit_vector())

                        end = array_point_to_image_pixel(laminar_axis.get_end_point())
                        end2 = array_point_to_image_pixel(laminar_axis_2.get_end_point())
                        start = array_point_to_image_pixel(laminar_axis.get_start_point())
                        start2 = array_point_to_image_pixel(laminar_axis_2.get_start_point())
                        la_corners[la_file]["line_loc"] = [start + end, start2 + end2]
                        la_corners[la_file]["text_loc"] = (int((end[0] + end2[0]) / 2), 
                                                           int((end[1] + end2[1]) / 2))


                images = []
                img_filenames = []
                for frame in range(start_frame, start_frame + movie_length, frame_step_size):
                    filename = output_dir + str(frame) + ".jpg"
                    if replace_existing_frame_pngs or not os.path.exists(filename):
                        # change frame
                        if not disable_photoZ_interact:
                            aPhz.set_measure_window(frame, None)

                            # export this frame
                            aPhz.save_map_jpeg(filename)
                            print("File created:", filename)

                    if annotations_enabled:
                        # draw electrode location on image <filename>
                        image = Image.open(filename)
                        draw = ImageDraw.Draw(image)
                        draw.ellipse(stim_xy, outline='red', width=2)
                        # read in corners files
                        for la_file in [corners_layer_file, corners_barrel_file]:
                            draw.line(la_corners[la_file]["line_loc"][0], fill='white', width=2)
                            draw.line(la_corners[la_file]["line_loc"][1], fill='white', width=2)
                            if la_file == corners_layer_file:
                                draw.text(la_corners[la_file]["text_loc"], "L4", anchor="ms")
                        # PIL image write back
                        image.save(filename)

                    try:
                        images.append(imageio.imread(filename))
                        img_filenames.append(filename)
                    except Exception as e:
                        pass

                # create gif
                created_movie = False
                try:
                    imageio.mimsave(output_dir + rec_id + 'movie.gif', images)
                    print("CREATED MOVIE:", rec_id + 'movie.gif')
                    created_movie = True
                    if clean_scratch_files:
                        for img_file in img_filenames:
                            os.remove(img_file)
                except Exception as e:
                    if not created_movie:
                        print("Not creating movie for " + rec_id)
                    print(e)

C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Anna-NBQX/07-15-24
02_01_01


Opening 02_01_01.zda
Saved intermed_snr_df file: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Anna-NBQX/07-15-24/selected_zda//Data.dat
Moved color upper bound from 1.0 to 44.86
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Anna-NBQX/07-15-24/selected_zda//analysis02_01_01/130.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Anna-NBQX/07-15-24/selected_zda//analysis02_01_01/134.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Anna-NBQX/07-15-24/selected_zda//analysis02_01_01/138.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Anna-NBQX/07-15-24/selected_zda//analysis02_01_01/142.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Anna-NBQX/07-15-24/selected_zda//analysis02_01_01/146.jpg
File created: C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Anna-NBQX/07-15-24/sele

## 